**<h2>PDF Reader with Llama 2 Model</h2>**

Necessary Libraries required for setup

- LLama Index: For Reading the PDF and creating vector embeddings for performing similarity search

- CTransformers - library for optimised approach for using LLM

- Sentence Transformers - to create sentence embeddings for each sentence

In [13]:
!pip install -q langchain ctransformers bitsandbytes pypdf python-dotenv einops accelerate

In [14]:
!pip install -q llama-index sentence_transformers

In [15]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [16]:
documents = SimpleDirectoryReader("/content/").load_data()
documents

[Document(id_='63dbb309-8a5b-4c5c-bb57-8456774498d3', embedding=None, metadata={'page_label': '1', 'file_name': 'Karnataka_Budget_Analysis_2023-24.pdf', 'file_path': '/content/Karnataka_Budget_Analysis_2023-24.pdf', 'file_type': 'application/pdf', 'file_size': 397902, 'creation_date': '2023-12-20', 'last_modified_date': '2023-12-20', 'last_accessed_date': '2023-12-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='cdb26b4b8d2341334d5015fb53031558f7b45362c36bb224c02bf90092d3af44', text=' \n \nTushar Chakrabarty  \ntushar@prsindia.org  Rutvik Upadhyaya  \nrutvik @prsindia.org  July 13, 2023  \nPRS Legislative Research  ◼ Institute for Policy Research Studies   \n3rd Floor, Gandharva Mahavidyalaya ◼ 212, Deen Dayal Upadhyaya Marg  ◼ New Delhi – 110002 \n

In [17]:
system_prompt = 'You are a QA Assistant. Your main goal is to answer the questions as correctly and promptly as possible based on the input and context provided'

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [18]:
from ctransformers import AutoConfig,AutoModelForCausalLM
config = AutoConfig.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML")
# Explicitly set the max_seq_len
config.max_seq_len = 4096
config.max_answer_len= 1024

llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q2_K.bin",config=config)
print(llm("""{Something with 3000 tokens}"""))

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]



For each of the three options, I have provided a different number of tokens. Please choose the option that requires the most tokens to complete.

Option 1: Create a new NFT collection with 5 rare and 5 common NFTs. Each rare NFT costs 50 tokens and each common NFT costs 20 tokens.

Option 2: Buy 5 exclusive NFTs, each of which costs 60 tokens.

Option 3: Purchase a rare NFT that currently owns an entire collection of NFTs worth 3000 tokens.

Which option do you prefer to choose and why? Please explain your reasoning.


In [19]:
# import torch

# llm = HuggingFaceLLM(
#     context_window = 4096,
#     max_new_tokens = 256,
#     generate_kwargs = {'temperature':0,'do_sample':False},
#     system_prompt = system_prompt,
#     query_wrapper_prompt = query_wrapper_prompt,
#     tokenizer_name = "llama-2-7b-chat.ggmlv3.q2_K.bi",
#     model_name = "llama-2-7b-chat.ggmlv3.q2_K.bi",
#     model_kwargs={'torch_dytpe':torch.float16}
# )

In [20]:
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

config = {'max_new_tokens': 500, 'context_length':4000}

llm = CTransformers(model = 'TheBloke/Llama-2-7B-Chat-GGML',model_file = 'llama-2-7b-chat.ggmlv3.q2_K.bin',callback = [StreamingStdOutCallbackHandler],
                    config = config)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-mpnet-base-v2')
)

In [22]:
service_context = ServiceContext.from_defaults(chunk_size = 2000, llm = llm,embed_model = embed_model)

In [23]:
index = VectorStoreIndex.from_documents(documents, service_context = service_context)

In [24]:
# query_engine = index.as_query_engine()
# response = query_engine.query("What are natural polymers?")
# response

In [25]:
# response.response

In [26]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the Gross State Domestic Product of Karnataka estimated to be?")
response.response

' The Gross State Domestic Product (GSDP) of Karnataka for 2023-24 is estimated to be Rs  25.7 lakh crore, amounting to a growth of 17.7% over the revised estimates of 2022 -23.'